In [24]:
from setup import D

synth_count = 4
d = D(
    synth_count=synth_count,
    # context='tyrelln6',
    context='opl',
)

# FIXME: doesn't work for 'opl' context with state=1 (state=0 works, state=2 works)
def finetune(state=0, synth_idx=0):
    match state:
        case 0:
            d.open_editor(synth_idx)
        case 1:
            # transfer instrument to all synths
            d.save_preset(synth_idx)
            import time
            time.sleep(0.5)
            for i in range(synth_count):
                d.load_preset(i)
        case _:
            d.print_params(synth_idx)


In [ ]:
def set_synths():
    for i in range(3):
        d.set_synth(i, 'sdrum2')
set_synths()

root_hz = 220.0
notes = []
harmonics = [1/4, 1/2, 1, 2, 4]
for h in harmonics:
    notes.append([root_hz * h])

finetune(0)
while True:
    d.play_non_edo(notes, synth_idx=[0, 1, 2], dur=0.12)
    # finetune(1)
    import time
    time.sleep(1.0)

KeyboardInterrupt: 

In [ ]:
finetune(2)

In [25]:
# todo: kick, open hi-hat, closed hi-hat

# Bass drum: Uses both operators. Essentially just doubles output amplitude?
# Hi-hat: Uses modulator settings. Half-sine recommended.

# Snare: Uses carrier settings. Abs-sine waveform recommended.
# Tom: Uses modulator settings. Sine waveform recommended.
# Cymbal: Uses carrier settings. Half-sine recommended.

from domblar.rhythm import rhythm_to_notes

# four on the floor

def set_synths():
    d.set_synth(0, 'bdrum_ok')
    d.set_synth(1, 'sdrum2')
    d.set_synth(2, 'h_hit1')
    d.set_synth(3, 'h_hit1')
set_synths()

edo = 2
scale = list(range(edo))

kick_voice = 'x...' * 4
kick_voice = rhythm_to_notes(kick_voice, -2 * edo)
print(kick_voice)
snare_voice = (['.'] * 4 + ['x', '.', '.', '.']) * 2
snare_voice = rhythm_to_notes(snare_voice, -4 * edo)
print(snare_voice)
closed_hihat_voice = ['.'] * 8 + ['.', 'x', '.', '.'] + ['.'] * 4
closed_hihat_voice = rhythm_to_notes(closed_hihat_voice, 0)
print(closed_hihat_voice)
open_hihat_voice = ['.', '.', 'x', '.'] * 4
open_hihat_voice = rhythm_to_notes(open_hihat_voice, -edo)
print(open_hihat_voice)
voices = list(zip(kick_voice, snare_voice, closed_hihat_voice, open_hihat_voice))
# voices = list(zip(kick_voice, open_hihat_voice))

for _ in range(4):
    d.play(voices, scale, edo, synth_idx=[0, 1, 2, 3], dur=0.12)


[-4, '.', '.', '.', -4, '.', '.', '.', -4, '.', '.', '.', -4, '.', '.', '.']
['.', '.', '.', '.', -8, '.', '.', '.', '.', '.', '.', '.', -8, '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', 0, '.', '.', '.', '.', '.', '.']
['.', '.', -2, '.', '.', '.', -2, '.', '.', '.', -2, '.', '.', '.', -2, '.']


KeyboardInterrupt: 

In [ ]:
import time

from domblar.rhythm import rhythm_to_notes
from domblar.edo import get_freq

# polyrhythms

def set_synths():
    d.set_synth(0, 'timpani2')
    d.set_synth(1, 'snare1')
    # d.set_synth(2, 'closed_hi_hat')
    # d.set_synth(3, 'hi_hat')
set_synths()

def play_events(d, voices, beat):
    # event: (note, edo, scale, dur: float, sus, amp, synth_idx)
    # voices: [[event]]
    # each idx is separate voice/channel

    last_event_idx = [0] * len(voices)
    last_event_time = [0] * len(voices)
    final_sleep = 0
    between_sleep = beat
    start_time = time.time()

    beats_ahead = 1  # FIXME: how much do we need this?

    progressed = True
    waiting = False
    while progressed or waiting:
        progressed = False
        waiting = False
        for voice_idx in range(len(voices)):
            if last_event_idx[voice_idx] >= len(voices[voice_idx]):
                continue
            waiting = True
            cur_beat = (time.time() - start_time) / beat
            if last_event_time[voice_idx] / beat > cur_beat + beats_ahead:
                continue

            note, edo, scale, dur, sus, amp, synth_idx = voices[voice_idx][last_event_idx[voice_idx]]
            freq = get_freq(note, scale, edo)
            cur_beat = (time.time() - start_time)
            if last_event_time[voice_idx] > cur_beat + beats_ahead * beat:
                continue
            send_note_dur = beat * sus
            time_increase = beat * dur
            timetag = start_time + last_event_time[voice_idx]
            between_sleep = min(between_sleep, time_increase)
            final_sleep = max(final_sleep, timetag + time_increase)

            d.client.send_note(
                synth_idx,
                freq=freq, dur=send_note_dur, amp=amp,
                timetag=timetag, channel=0)  # TODO: for MPE with channels use channel=note_idx

            last_event_idx[voice_idx] += 1
            last_event_time[voice_idx] += time_increase
            progressed = True

        if waiting and not progressed:
            # sleep smallest beat division that we observed so far
            time.sleep(between_sleep)

    time.sleep(final_sleep - time.time())


edo = 2
scale = list(range(edo))

events = [[], []]

# TODO: replace with 'x...' * 4
kick_voice = 'x..x..x..x.' * 2  # 22 notes
kick_voice = rhythm_to_notes(kick_voice, 0)
print(kick_voice)
kick_dur = 1
kick_sum_dur = 0
for rep in range(10):
    for note in kick_voice:
        dur = kick_dur / len(kick_voice)
        kick_sum_dur += dur
        sus = dur * 0.8
        amp = 1
        synth_idx = voice_idx = 0
        events[voice_idx].append((note, edo, scale, dur, sus, amp, synth_idx))


# snare_voice = '.x.x' * 5  # 20 notes
snare_voice = 'x'  # 1 note
snare_voice = rhythm_to_notes(snare_voice, -4 * edo)
print(snare_voice)
snare_sum_dur = 0
for rep in range(11):
    for note in snare_voice:
        dur = kick_dur * 20/22 / len(snare_voice)
        snare_sum_dur += dur
        sus = dur * 0.5
        amp = 1
        synth_idx = voice_idx = 1
        events[voice_idx].append((note, edo, scale, dur, sus, amp, synth_idx))

# print('durs', kick_sum_dur, snare_sum_dur)

# closed_hihat_voice = ['.'] * 8 + ['.', 'x', '.', '.'] + ['.'] * 4
# closed_hihat_voice = rhythm_to_notes(closed_hihat_voice, -edo)
# print(closed_hihat_voice)
# open_hihat_voice = ['.', '.', 'x', '.'] * 4
# open_hihat_voice = rhythm_to_notes(open_hihat_voice, -8 * edo)
# print(open_hihat_voice)
# voices = list(zip(kick_voice, snare_voice, closed_hihat_voice, open_hihat_voice))

# for _ in range(4):
play_events(d, events, beat=1.5)

# TODO:
# - mini-notation for polyrhythms
# - support several streams
# - rational numbers
# - understanding what is playing concurrently and
# coordinating compatibility of simultaneous sounds
# - sync streams



[0, '.', '.', 0, '.', '.', 0, '.', '.', 0, '.', 0, '.', '.', 0, '.', '.', 0, '.', '.', 0, '.']
[-8]


KeyboardInterrupt: 

[-2, '.', '.', '.', -2, '.', '.', '.', -2, '.', '.', '.', -2, '.', '.', '.']
['.', '.', '.', '.', -8, '.', '.', '.', '.', '.', '.', '.', -8, '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.', -2, '.', '.', '.', '.', '.', '.']
['.', '.', -16, '.', '.', '.', -16, '.', '.', '.', -16, '.', '.', '.', -16, '.']


In [ ]:
# the levee break

def set_synths():
    d.set_synth(0, 'kick03')
    d.set_synth(1, 'log_drum')
    d.set_synth(2, 'closed_hi_hat')
set_synths()

edo = 2
scale = list(range(edo))

kick_voice = ['x', 'x', '.', '.'] + ['.', '.', '.', 'x'] + ['.', '.', 'x', 'x'] + ['.'] * 4
kick_voice = rhythm_to_notes(kick_voice, 0)
print(kick_voice)
snare_voice = (['.'] * 4 + ['x', '.', '.', '.']) * 2
snare_voice = rhythm_to_notes(snare_voice, edo)
print(snare_voice)
closed_hihat_voice = ['x', '.'] * 8
closed_hihat_voice = rhythm_to_notes(closed_hihat_voice, -8*edo)
print(closed_hihat_voice)
voices = list(zip(kick_voice, snare_voice, closed_hihat_voice))


for _ in range(4):
    d.play(voices, scale, edo, synth_idx=[0, 1, 2], dur=0.15)


[0, 0, '.', '.', '.', '.', '.', 0, '.', '.', 0, 0, '.', '.', '.', '.']
['.', '.', '.', '.', 2, '.', '.', '.', '.', '.', '.', '.', 2, '.', '.', '.']
[-16, '.', -16, '.', -16, '.', -16, '.', -16, '.', -16, '.', -16, '.', -16, '.']


In [ ]:
# FIXME: replace with method/function
finetuning = 0
synth_idx = 0

try:
    finetuning
except Exception as e:
    d.open_editor(synth_idx)
    finetuning = 1
else:
    if finetuning == 0:
        d.open_editor(synth_idx)
    elif finetuning == 1:
        # transfer instrument to all synths
        d.save_preset(synth_idx)
        import time
        time.sleep(0.5)
        for i in range(synth_count):
            d.load_preset(i)
    else:
        d.print_params(synth_idx)

In [ ]:
# cleanup
d.stop_server()